<a href="https://colab.research.google.com/github/geraldmc/DL4CV-2022/blob/main/test_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Standard libraries
import os
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
try:
  from tqdm.notebook import tqdm
except ImportError:
  print("Unable to import tqdm library.")

In [ ]:
import torch
print("Using torch", torch.__version__)

# Note on Mac M1 this prints ... 'Using torch 1.9.1.post3'

Using torch 1.10.0+cu111


In [ ]:
gpu_avail = torch.cuda.is_available()
if gpu_avail:
  print(f"Is the GPU available: {gpu_avail}")
  print("GPU type: {}".format(torch.cuda.get_device_name(0)))
else:
  print("GPU unavailable.") 

Is the GPU available: True
GPU type: Tesla P100-PCIE-16GB


In [ ]:
import cv2
cv2.__version__

'4.1.2'

In [ ]:
import torchvision
torchvision.__version__

'0.11.1+cu111'

In [ ]:
import PIL
PIL.__version__

'7.1.2'

The following uses 'imgaug' for augmentation: 
https://medium.com/pytorch/ai-for-ag-production-machine-learning-for-agriculture-e8cfdb9849a1

In [ ]:
from imgaug import augmenters as iaa
import os
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
from torchvision import datasets, transforms

DATA_DIR = './fashionMNIST/'

class CustomAugmentor:
    
    def __init__(self):
            self.aug = iaa.Sequential([iaa.flip.Fliplr(p=0.5),
                       iaa.GaussianBlur(sigma=(0.0, 0.1)),
                       iaa.Multiply((0.9, 1.1)),
                       iaa.Dropout((0, 0.05)),
                       iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)) ])
    
    def __call__(self, img):
        img = np.array(img)
        # Return a copy here to work around the error: 
        # ValueError: At least one stride in the given numpy array is negative, 
        # and tensors with negative strides are not currently supported.
        return self.aug.augment_image(img).copy()

# transforms for images
transform=transforms.Compose([CustomAugmentor(), transforms.ToTensor()])
fmnist_train = FashionMNIST(DATA_DIR, train=True, download=True, transform=transform)
fmnist_test = FashionMNIST(DATA_DIR, train=False, download=True, transform=transforms.ToTensor())
fmnist_train, fmnist_val = random_split(fmnist_train, [55000, 5000])

train_dl = DataLoader(fmnist_train, batch_size=64)
val_dl = DataLoader(fmnist_val, batch_size=64)
test_dl = DataLoader(fmnist_test, batch_size=64)